## imports

In [1]:
from multiprocessing import Event, Pool, Queue
import time

import cv2
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [2]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
fps = 60
frame_buf_maxsize = 5

## cf helper functions

In [3]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)
    return

## init

In [4]:
def init_cf():
    # Initialize all the CrazyFlie drivers:
    cflib.crtp.init_drivers(enable_debug_driver=False)

    # Scan for Crazyflies in range of the antenna:
    print('Scanning interfaces for Crazyflies...')
    available = cflib.crtp.scan_interfaces()

    # List local CrazyFlie devices:
    print('Crazyflies found:')
    for i in available:
        print(i[0])
        
    if len(available) == 0:
        try:
            raise Exception('No Crazyflies found, cannot run example')
        except:
            return None
        
    scf = SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache'))
    
    return scf.cf

def init(f_b, st):
    global frame_buf, stop, cf, failed
    frame_buf = f_b
    stop = st
    cf = init_cf()
    failed = False
    
    if cf is None:
        failed = True

## video capture

In [5]:
def capture_frames(camera, fps):
    if failed:
        return
    
    cap = cv2.VideoCapture(camera)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    while not stop.is_set():
        ok, frame = cap.read()

        if ok:
            # TODO: make sure queue doesnt fill up
            frame_buf.put_nowait(frame)
        else:
            break

        time.sleep(1 / fps)
            
    cap.release()
    cv2.destroyAllWindows()

## planning

In [6]:
def process_frame():
    if failed:
        return
    # read frame and return next position
    pass

## control drone

In [7]:
def send_control():
    if failed:
        return
    # send control input to drone
    pass

## main loop

In [8]:
# TEMPORARY
def f():
    stop.set()

In [9]:
frame_buf = Queue(frame_buf_maxsize)
stop = Event()

pool = Pool(6, initializer=init, initargs=(frame_buf, stop))

capture_future = pool.apply_async(capture_frames, args=(camera, fps))
time.sleep(1)
f_future = pool.apply_async(f)

pool.close()
pool.join()

Scanning interfaces for Crazyflies...
Scanning interfaces for Crazyflies...
Scanning interfaces for Crazyflies...Cannot find a Crazyradio Dongle

Scanning interfaces for Crazyflies...

Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Cannot find a Crazyradio Dongle
Crazyflies found:
Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Cannot find a Crazyradio Dongle
Crazyflies found:

Scanning interfaces for Crazyflies...


...

Crazyflies found:Cannot find a Crazyradio Dongle



Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Cannot find a Crazyradio DongleCrazyflies found:



Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Crazyflies found:
